<a href="https://colab.research.google.com/github/dudl1/FamilyService/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://deb.nodesource.com/setup_22.x | sudo bash -
!sudo apt-get install -y nodejs
!node -v
!npm install puppeteer

In [ ]:
!npm install node-fetch@2

In [86]:
%%writefile search.js
const puppeteer = require('puppeteer')
const fetch = require('node-fetch') // если у тебя Node <18, иначе можно использовать глобальный fetch

async function getCoordsFromOSM(cityName) {
  const url = `https://nominatim.openstreetmap.org/search?format=json&q=${encodeURIComponent(cityName)}`
  const response = await fetch(url)
  const data = await response.json()

  if (data.length === 0) {
    throw new Error('Город не найден в OSM')
  }

  const place = data[0]
  return {
    latitude: parseFloat(place.lat),
    longitude: parseFloat(place.lon)
  }
}

function createYandexTrafficUrl(city, longitude, latitude, zoom = 12) {
  const coords = encodeURIComponent(`${longitude},${latitude}`)
  return `https://yandex.ru/maps/62/${city}/probki/?ll=${coords}&z=${zoom}`
}

async function fetchYandexTraffic(browser, url) {
  const page = await browser.newPage()
  await page.goto(url, { waitUntil: 'networkidle2' })

  try {
    await page.waitForSelector('.card-feature-view .traffic-icon', { timeout: 15000 })

    const traffic = await page.evaluate(() => {
      const scoreEl = document.querySelector('.card-feature-view__icon-wrapper .traffic-icon__text')
      const score = scoreEl ? scoreEl.innerText.trim() : null

      const hourEls = document.querySelectorAll('.traffic-forecast-view__hour')
      const hours = Array.from(hourEls).map(el => {
        const levelEl = el.querySelector('.traffic-forecast-view__hour-level')
        const textEl = el.querySelector('.traffic-forecast-view__hour-text')
        return {
          hour: textEl ? textEl.innerText : null,
          level: levelEl ? levelEl.innerText : null
        }
      })

      let homeRoute = null
      const homeBtn = document.querySelector('.home-work-route-view__button-text')
      if (homeBtn) {
        const match = homeBtn.innerText.match(/(\d+)\s*мин/)
        if (match) {
          homeRoute = parseInt(match[1], 10)
        }
      }

      return { score, hours, homeRoute }
    })

    await page.close()
    return traffic

  } catch (err) {
    console.error('Ошибка при получении данных:', err.message)
    await page.close()
    return null
  }
}

async function main() {
  const browser = await puppeteer.launch({
    headless: true,
    args: ['--no-sandbox']
  })

  try {
    const cityName = 'Санкт-Петербург'       // название города для OSM
    const yandexCityName = 'sankt-peterburg' // название города в URL Яндекса (можно в нижнем регистре и через дефисы)

    console.time('OSM geocoding')
    const coords = await getCoordsFromOSM(cityName)
    console.timeEnd('OSM geocoding')
    console.log(`Координаты города ${cityName}:`, coords)

    const url = createYandexTrafficUrl(yandexCityName, coords.longitude, coords.latitude)
    console.log('URL для Яндекс.Карт:', url)

    console.time('Fetch Yandex traffic')
    const data = await fetchYandexTraffic(browser, url)
    console.timeEnd('Fetch Yandex traffic')

    if (!data) {
      console.log('Не удалось получить данные о пробках')
      return
    }

    console.log(`🔴 Пробки сейчас в ${cityName}: ${data.score} баллов`)
    console.log('🕒 Прогноз пробок по часам:')
    for (const h of data.hours) {
      console.log(`  – ${h.hour}:00 → ${h.level} баллов`)
    }
    if (data.homeRoute !== null) {
      console.log(`🏠 Время: домой — ${data.homeRoute} мин`)
    }

  } catch (error) {
    console.error('Ошибка:', error.message)
  } finally {
    await browser.close()
  }
}

main()

Overwriting search.js


In [87]:
!node search.js

OSM geocoding: 416.871ms
Координаты города Санкт-Петербург: { latitude: 59.9606739, longitude: 30.1586551 }
URL для Яндекс.Карт: https://yandex.ru/maps/62/sankt-peterburg/probki/?ll=30.1586551%2C59.9606739&z=12
Fetch Yandex traffic: 6.224s
🔴 Пробки сейчас в Санкт-Петербург: 1 баллов
🕒 Прогноз пробок по часам:
  – 4:00 → 0 баллов
  – 5:00 → 0 баллов
  – 6:00 → 0 баллов
  – 7:00 → 1 баллов
  – 8:00 → 3 баллов
  – 9:00 → 3 баллов
